In [60]:
z = h0 - 0.25*m
evals, evecs = jnp.linalg.eigh(z)
expevals = jnp.exp(evals)

In [62]:
jnp.linalg.norm(z - evecs @ jnp.diag(evals) @ evecs.conj().T)

Array(3.58514592e-15, dtype=float64)

In [66]:
jnp.linalg.norm(evecs @ evecs.conj().T - jnp.eye(n))

Array(1.33191522e-15, dtype=float64)

In [68]:
a = evecs.conj().T @ m @ evecs

In [154]:
evals1, evals2 = jnp.meshgrid(evals, evals, indexing='ij')
mask = jnp.ones((n,n)) - jnp.eye(n)
expevals1, expevals2 = jnp.meshgrid(expevals, expevals, indexing='ij')

In [159]:
diagterm1 = expevals1*jnp.diag(jnp.diag(a*a))
print(diagterm1)

[[8.19864611e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.74939119e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.33854857e-03]]


In [162]:
testmat3 = np.zeros((n,n))
for i in range(3):
    for k in range(3):
        if i==k:
            testmat3[i,k] = 0.5*expevals[i]*(a[i,i]*a[i,k])*2

print(jnp.linalg.norm(diagterm1-testmat3))

0.0


In [153]:
numer1 = -expevals1 + evals1*expevals1 - evals2*expevals1 + expevals2
denom1 = (evals1-evals2)**2 * mask + jnp.eye(n)
frac1 = numer1/denom1 * mask
term1 = frac1*2*(jnp.diag(a)*a).T

In [89]:
testmat2

array([[ 8.61595771e+00, -5.37480823e+00,  3.65436632e+00],
       [ 1.25177075e+00,  4.67333816e-01, -8.74234820e-01],
       [ 1.37752824e-02,  1.41499436e-02,  1.22427907e-04]])

In [94]:
2*(jnp.diag(a)*a).T

Array([[ 8.61595771e+00, -5.37480823e+00,  3.65436632e+00],
       [ 1.25177075e+00,  4.67333816e-01, -8.74234820e-01],
       [ 1.37752824e-02,  1.41499436e-02,  1.22427907e-04]],      dtype=float64)

In [102]:
testmat = np.zeros((n,n))
testmat2 = np.zeros((n,n))
testmat3 = np.zeros((n,n))
for i in range(3):
    for k in range(3):
        if i != k:
            testmat[i,k] = (-expevals[i]+evals[i]*expevals[i]-evals[k]*expevals[i]+expevals[k])/(evals[i]-evals[k])**2
        testmat2[i,k] = a[i,i]*a[i,k] + a[i,i]*a[i,k]
        testmat3[i,k] = testmat[i,k] * testmat2[i,k]
print(jnp.linalg.norm(term1-testmat3))

1.831026719408895e-15


In [101]:
numer2 = -expevals1 + evals1*expevals2 - evals2*expevals2 + expevals2
denom2 = (evals1-evals2)**2 * mask + jnp.eye(n)
frac2 = numer2/denom2 * mask
term2 = frac2*2*(a*jnp.diag(a))

In [103]:
testmat = np.zeros((n,n))
testmat2 = np.zeros((n,n))
testmat3 = np.zeros((n,n))
for i in range(3):
    for k in range(3):
        if i != k:
            testmat[i,k] = (-expevals[i]+evals[i]*expevals[k]-evals[k]*expevals[k]+expevals[k])/(evals[i]-evals[k])**2
        testmat2[i,k] = a[i,k]*a[k,k] + a[i,k]*a[k,k]
        testmat3[i,k] = testmat[i,k] * testmat2[i,k]
print(jnp.linalg.norm(term2-testmat3))

0.0


In [104]:
paren = 2*jnp.einsum('ij,jk->ijk',a,a)

In [146]:
evals1, evals2, evals3 = jnp.meshgrid(evals, evals, evals, indexing='ij')
expevals1, expevals2, expevals3 = jnp.meshgrid(expevals, expevals, expevals, indexing='ij')

numer3 = evals1*(expevals2-expevals3)-evals2*(expevals1-expevals3)+evals3*(expevals1-expevals2)

mask = np.zeros((3,3,3))
for i in range(3):
    for j in range(3):
        for k in range(3):
            if i!=j and i!=k and j!=k:
                mask[i,j,k] = 1

mask = jnp.array(mask)
# print(mask)

denom3 = (evals1-evals2)*(evals1-evals3)*(evals2-evals3) + jnp.ones((3,3,3)) - mask

term3 = jnp.sum( (numer3/denom3 * mask) * paren, axis=1 )

In [147]:
testmat3 = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        for k in range(3):
            if i!=j and i!=k and j!=k:
                fractermN = evals[i]*(expevals[j]-expevals[k])-evals[j]*(expevals[i]-expevals[k])+evals[k]*(expevals[i]-expevals[j])
                fractermD = (evals[i]-evals[j])*(evals[i]-evals[k])*(evals[j]-evals[k])
                fracterm = fractermN/fractermD
                parenterm = a[i,j]*a[j,k]*2
                testmat3[i,k] += fracterm*parenterm

In [148]:
print(jnp.linalg.norm(term3-testmat3))

8.881784197001252e-16


In [ ]:
newmask = np.zeros((3,3,3))
for i in range(3):
    for j in range(3):
        for k in range(3):
            if i!=j and i==k:
                newmask[i,j,k] = 1

diagnumer2 = -expevals1 + evals1*expevals1 - evals2*expevals1 + expevals2
diagdenom2 = (evals1-evals2)**2 + jnp.ones((3,3,3)) - newmask
diagfrac2 = (diagnumer2/diagdenom2)*newmask
diagterm2 = jnp.sum( diagfrac2*paren, axis=1 )
print(diagterm2)

[[ 9.30389424  0.          0.        ]
 [ 0.         17.39513295  0.        ]
 [ 0.          0.         21.58479317]]


In [149]:
testmat3 = np.zeros((3,3))
for i in range(3):
    for j in range(3):
        for k in range(3):
            if i!=j and i==k:
                fractermN = -expevals[i] + evals[i]*expevals[i] - evals[j]*expevals[i] + expevals[j]
                fractermD = (evals[i]-evals[j])**2
                fracterm = fractermN/fractermD
                parenterm = 2*a[i,j]*a[j,k]
                testmat3[i,k] += fracterm*parenterm

In [151]:
print(jnp.linalg.norm(diagterm2-testmat3))

7.105427357601002e-15


In [166]:
r = evecs @ (term1 - term2 - term3 + diagterm1 + diagterm2) @ evecs.conj().T
print(r)

[[ 19.48758841   3.31180876  -4.46688522]
 [  3.31180876  21.28372786 -12.29398882]
 [ -4.46688522 -12.29398882  17.46187993]]


In [168]:
ddmyexp(-0.25).reshape((n,n))

Array([[ 19.48758841,   3.31180876,  -4.46688522],
       [  3.31180876,  21.28372786, -12.29398882],
       [ -4.46688522, -12.29398882,  17.46187993]],      dtype=float64, weak_type=True)

In [169]:
print(jnp.linalg.norm(r - ddmyexp(-0.25).reshape((n,n))))

4.245176955060179e-14
